In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Contoh Penggunaan Sistem Pengenalan Wajah dan Deteksi Suku/Etnis\n",
    "\n",
    "Notebook ini menunjukkan cara menggunakan fungsi utama dari sistem pengenalan wajah, deteksi suku/etnis, dan deteksi gender tanpa menjalankan aplikasi Streamlit. Kita akan mencoba:\n",
    "- **Deteksi Wajah**: Menggunakan MTCNN, RetinaFace, atau Haar Cascade.\n",
    "- **Face Similarity**: Membandingkan kemiripan wajah menggunakan Siamese Network, FaceNet, atau ArcFace.\n",
    "- **Klasifikasi Suku/Etnis**: Menggunakan model ResNet50 untuk memprediksi suku/etnis.\n",
    "- **Deteksi Gender**: Menggunakan DeepFace untuk memprediksi gender.\n",
    "\n",
    "## Prasyarat\n",
    "- Pastikan semua dependensi di `requirements.txt` telah terinstal.\n",
    "- Siapkan gambar wajah untuk pengujian.\n",
    "- Model `ethnic_model.pth` dan `siamese_model.h5` harus tersedia (dihasilkan dari pelatihan di aplikasi Streamlit)."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Impor Dependensi\n",
    "\n",
    "Mari kita impor semua pustaka yang diperlukan."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import cv2\n",
    "import numpy as np\n",
    "import torch\n",
    "import torch.nn as nn\n",
    "from facenet_pytorch import InceptionResnetV1\n",
    "from torchvision import models, transforms\n",
    "import matplotlib.pyplot as plt\n",
    "import tensorflow as tf\n",
    "from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense\n",
    "from tensorflow.keras.models import Model\n",
    "from scipy.spatial.distance import cosine\n",
    "import os\n",
    "import tempfile\n",
    "from deepface import DeepFace\n",
    "\n",
    "# Coba impor MTCNN\n",
    "try:\n",
    "    from mtcnn import MTCNN\n",
    "    MTCNN_AVAILABLE = True\n",
    "    print(\"MTCNN berhasil diimpor!\")\n",
    "except ImportError:\n",
    "    MTCNN_AVAILABLE = False\n",
    "    print(\"MTCNN tidak tersedia. Menggunakan Haar Cascade.\")\n",
    "\n",
    "# Coba impor RetinaFace\n",
    "try:\n",
    "    from retinaface import RetinaFace\n",
    "    RETINAFACE_AVAILABLE = True\n",
    "    print(\"RetinaFace berhasil diimpor!\")\n",
    "except ImportError:\n",
    "    RETINAFACE_AVAILABLE = False\n",
    "    print(\"RetinaFace tidak tersedia. Menggunakan MTCNN atau Haar Cascade.\")\n",
    "\n",
    "# Inisialisasi FaceNet\n",
    "FACENET_AVAILABLE = True\n",
    "try:\n",
    "    facenet_model = InceptionResnetV1(pretrained='vggface2').eval()\n",
    "    if torch.cuda.is_available():\n",
    "        facenet_model = facenet_model.cuda()\n",
    "    print(\"FaceNet model berhasil diimpor!\")\n",
    "except Exception as e:\n",
    "    FACENET_AVAILABLE = False\n",
    "    print(f\"Gagal mengimpor FaceNet: {str(e)}\")\n",
    "\n",
    "# Inisialisasi ArcFace\n",
    "ARCFACE_AVAILABLE = True\n",
    "try:\n",
    "    arcface_model = models.resnet50(weights='IMAGENET1K_V1')\n",
    "    arcface_model.fc = nn.Linear(arcface_model.fc.in_features, 512)\n",
    "    if torch.cuda.is_available():\n",
    "        arcface_model = arcface_model.cuda()\n",
    "    arcface_model.eval()\n",
    "    print(\"ArcFace model (ResNet50) berhasil diimpor!\")\n",
    "except Exception as e:\n",
    "    ARCFACE_AVAILABLE = False\n",
    "    print(f\"Gagal mengimpor ArcFace: {str(e)}\")\n",
    "\n",
    "# Fungsi preprocessing gambar untuk FaceNet\n",
    "def preprocess_image_for_facenet(image, size=(160, 160)):\n",
    "    image = cv2.resize(image, size)\n",
    "    image = image.astype(np.float32)\n",
    "    image = (image - 127.5) / 128.0\n",
    "    image = np.transpose(image, (2, 0, 1))\n",
    "    image = torch.tensor(image).unsqueeze(0)\n",
    "    if torch.cuda.is_available():\n",
    "        image = image.cuda()\n",
    "    return image\n",
    "\n",
    "# Fungsi preprocessing gambar untuk Siamese\n",
    "def preprocess_image_for_siamese(image, size=(160, 160)):\n",
    "    image = cv2.resize(image, size)\n",
    "    image = image.astype(np.float32) / 255.0\n",
    "    return image\n",
    "\n",
    "# Fungsi preprocessing gambar untuk ArcFace\n",
    "def preprocess_image_for_arcface(image, size=(224, 224)):\n",
    "    image = cv2.resize(image, size)\n",
    "    transform = transforms.Compose([\n",
    "        transforms.ToPILImage(),\n",
    "        transforms.ToTensor(),\n",
    "        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])\n",
    "    ])\n",
    "    image = transform(image).unsqueeze(0)\n",
    "    if torch.cuda.is_available():\n",
    "        image = image.cuda()\n",
    "    return image\n",
    "\n",
    "# Fungsi mendapatkan embedding wajah\n",
    "def get_face_embedding_facenet(image):\n",
    "    if not FACENET_AVAILABLE:\n",
    "        print(\"FaceNet tidak tersedia.\")\n",
    "        return None\n",
    "    processed_image = preprocess_image_for_facenet(image)\n",
    "    if processed_image is None:\n",
    "        return None\n",
    "    with torch.no_grad():\n",
    "        embedding = facenet_model(processed_image).cpu().numpy()\n",
    "    return embedding.flatten()\n",
    "\n",
    "def get_face_embedding_siamese(image, siamese_model):\n",
    "    processed_image = preprocess_image_for_siamese(image)\n",
    "    if processed_image is None:\n",
    "        return None\n",
    "    processed_image = np.expand_dims(processed_image, axis=0)\n",
    "    embedding = siamese_model.predict(processed_image, verbose=0)\n",
    "    return embedding.flatten()\n",
    "\n",
    "def get_face_embedding_arcface(image):\n",
    "    if not ARCFACE_AVAILABLE:\n",
    "        print(\"ArcFace tidak tersedia.\")\n",
    "        return None\n",
    "    processed_image = preprocess_image_for_arcface(image)\n",
    "    if processed_image is None:\n",
    "        return None\n",
    "    with torch.no_grad():\n",
    "        embedding = arcface_model(processed_image).cpu().numpy()\n",
    "    return embedding.flatten()\n",
    "\n",
    "# Fungsi deteksi wajah dengan Haar Cascade\n",
    "def detect_faces_haar(image):\n",
    "    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')\n",
    "    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\n",
    "    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(30, 30))\n",
    "    return [(x, y, w, h) for (x, y, w, h) in faces]\n",
    "\n",
    "# Fungsi deteksi wajah dengan MTCNN\n",
    "def detect_faces_mtcnn(image):\n",
    "    if not MTCNN_AVAILABLE:\n",
    "        return detect_faces_haar(image)\n",
    "    detector = MTCNN(min_face_size=20, steps_threshold=[0.6, 0.7, 0.7])\n",
    "    faces = detector.detect_faces(image)\n",
    "    return [(face['box'][0], face['box'][1], face['box'][2], face['box'][3]) for face in faces]\n",
    "\n",
    "# Fungsi deteksi wajah dengan RetinaFace\n",
    "def detect_faces_retina(image):\n",
    "    if not RETINAFACE_AVAILABLE:\n",
    "        return detect_faces_mtcnn(image)\n",
    "    faces = RetinaFace.detect_faces(image)\n",
    "    adjusted_faces = []\n",
    "    for key in faces.keys():\n",
    "        face = faces[key]\n",
    "        x, y, w, h = face['facial_area']\n",
    "        adjusted_faces.append((x, y, w-x, h-y))\n",
    "    return adjusted_faces\n",
    "\n",
    "# Fungsi deteksi wajah utama\n",
    "def detect_faces(image, method='retina'):\n",
    "    if method == 'retina' and RETINAFACE_AVAILABLE:\n",
    "        faces = detect_faces_retina(image)\n",
    "    elif method == 'mtcnn' and MTCNN_AVAILABLE:\n",
    "        faces = detect_faces_mtcnn(image)\n",
    "    else:\n",
    "        faces = detect_faces_haar(image)\n",
    "    return faces\n",
    "\n",
    "# Fungsi preprocessing untuk klasifikasi suku\n",
    "def preprocess_for_classification(image):\n",
    "    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n",
    "    transform = transforms.Compose([\n",
    "        transforms.ToPILImage(),\n",
    "        transforms.Resize((224, 224)),\n",
    "        transforms.ToTensor(),\n",
    "        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])\n",
    "    ])\n",
    "    return transform(image).unsqueeze(0)\n",
    "\n",
    "# Inisialisasi model ResNet50 untuk klasifikasi suku\n",
    "def initialize_ethnic_model(num_classes):\n",
    "    model = models.resnet50(weights='IMAGENET1K_V1')\n",
    "    model.fc = nn.Sequential(\n",
    "        nn.Linear(model.fc.in_features, 1024),\n",
    "        nn.ReLU(),\n",
    "        nn.Dropout(0.5),\n",
    "        nn.Linear(1024, num_classes)\n",
    "    )\n",
    "    return model\n",
    "\n",
    "# Inisialisasi Siamese Network\n",
    "def create_base_network(input_shape=(160, 160, 3)):\n",
    "    model = tf.keras.Sequential([\n",
    "        Conv2D(64, (10, 10), activation='relu', input_shape=input_shape),\n",
    "        MaxPooling2D(),\n",
    "        Conv2D(128, (7, 7), activation='relu'),\n",
    "        MaxPooling2D(),\n",
    "        Conv2D(128, (4, 4), activation='relu'),\n",
    "        MaxPooling2D(),\n",
    "        Conv2D(256, (4, 4), activation='relu'),\n",
    "        Flatten(),\n",
    "        Dense(4096, activation='relu'),\n",
    "        Dense(128, activation='sigmoid')\n",
    "    ])\n",
    "    return model\n",
    "\n",
    "def create_siamese_model(input_shape=(160, 160, 3)):\n",
    "    base_network = create_base_network(input_shape)\n",
    "    return base_network\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Deteksi Wajah\n",
    "\n",
    "Mari kita coba mendeteksi wajah pada sebuah gambar menggunakan MTCNN, RetinaFace, atau Haar Cascade."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Ganti 'path_to_image.jpg' dengan path ke gambar Anda\n",
    "image_path = 'path_to_image.jpg'\n",
    "image = cv2.imread(image_path)\n",
    "if image is None:\n",
    "    print(\"Gagal memuat gambar. Pastikan path benar.\")\n",
    "else:\n",
    "    # Deteksi wajah\n",
    "    faces = detect_faces(image)\n",
    "    print(f\"Jumlah wajah yang terdeteksi: {len(faces)}\")\n",
    "\n",
    "    # Gambar kotak di sekitar wajah\n",
    "    image_with_faces = image.copy()\n",
    "    for (x, y, w, h) in faces:\n",
    "        cv2.rectangle(image_with_faces, (x, y), (x+w, y+h), (0, 255, 0), 2)\n",
    "\n",
    "    # Tampilkan gambar\n",
    "    plt.figure(figsize=(8, 6))\n",
    "    plt.imshow(cv2.cvtColor(image_with_faces, cv2.COLOR_BGR2RGB))\n",
    "    plt.axis('off')\n",
    "    plt.title('Hasil Deteksi Wajah')\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Face Similarity\n",
    "\n",
    "Mari kita bandingkan kemiripan antara dua wajah menggunakan Siamese Network, FaceNet, dan ArcFace. Anda perlu menyiapkan dua gambar wajah."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Ganti path berikut dengan path ke dua gambar Anda\n",
    "image_path1 = 'path_to_image1.jpg'\n",
    "image_path2 = 'path_to_image2.jpg'\n",
    "\n",
    "image1 = cv2.imread(image_path1)\n",
    "image2 = cv2.imread(image_path2)\n",
    "\n",
    "if image1 is None or image2 is None:\n",
    "    print(\"Gagal memuat salah satu gambar. Pastikan path benar.\")\n",
    "else:\n",
    "    # Deteksi wajah pada kedua gambar\n",
    "    faces1 = detect_faces(image1)\n",
    "    faces2 = detect_faces(image2)\n",
    "\n",
    "    if len(faces1) > 0 and len(faces2) > 0:\n",
    "        (x1, y1, w1, h1) = faces1[0]\n",
    "        (x2, y2, w2, h2) = faces2[0]\n",
    "        face1 = image1[y1:y1+h1, x1:x1+w1]\n",
    "        face2 = image2[y2:y2+h2, x2:x2+w2]\n",
    "\n",
    "        # Tampilkan wajah yang terdeteksi\n",
    "        plt.figure(figsize=(10, 5))\n",
    "        plt.subplot(1, 2, 1)\n",
    "        plt.imshow(cv2.cvtColor(face1, cv2.COLOR_BGR2RGB))\n",
    "        plt.title('Wajah 1')\n",
    "        plt.axis('off')\n",
    "        plt.subplot(1, 2, 2)\n",
    "        plt.imshow(cv2.cvtColor(face2, cv2.COLOR_BGR2RGB))\n",
    "        plt.title('Wajah 2')\n",
    "        plt.axis('off')\n",
    "        plt.show()\n",
    "\n",
    "        # Inisialisasi Siamese model\n",
    "        siamese_base_model = None\n",
    "        siamese_model_path = 'siamese_model.h5'\n",
    "        if os.path.exists(siamese_model_path):\n",
    "            siamese_base_model = create_siamese_model()\n",
    "            siamese_base_model.load_weights(siamese_model_path)\n",
    "            print(\"Siamese model berhasil dimuat!\")\n",
    "        else:\n",
    "            print(\"Siamese model belum dilatih. Silakan latih model terlebih dahulu di aplikasi Streamlit.\")\n",
    "\n",
    "        # Bandingkan wajah menggunakan tiga metode\n",
    "        methods = ['siamese', 'facenet', 'arcface']\n",
    "        for method in methods:\n",
    "            if method == 'siamese' and siamese_base_model is None:\n",
    "                print(\"Melewati Siamese karena model belum dimuat.\")\n",
    "                continue\n",
    "            if method == 'facenet' and not FACENET_AVAILABLE:\n",
    "                print(\"Melewati FaceNet karena tidak tersedia.\")\n",
    "                continue\n",
    "            if method == 'arcface' and not ARCFACE_AVAILABLE:\n",
    "                print(\"Melewati ArcFace karena tidak tersedia.\")\n",
    "                continue\n",
    "\n",
    "            # Hitung embedding\n",
    "            if method == 'facenet':\n",
    "                embedding1 = get_face_embedding_facenet(face1)\n",
    "                embedding2 = get_face_embedding_facenet(face2)\n",
    "            elif method == 'siamese':\n",
    "                embedding1 = get_face_embedding_siamese(face1, siamese_base_model)\n",
    "                embedding2 = get_face_embedding_siamese(face2, siamese_base_model)\n",
    "            elif method == 'arcface':\n",
    "                embedding1 = get_face_embedding_arcface(face1)\n",
    "                embedding2 = get_face_embedding_arcface(face2)\n",
    "\n",
    "            if embedding1 is not None and embedding2 is not None:\n",
    "                similarity = 1 - cosine(embedding1, embedding2)\n",
    "                print(f\"Skor Kemiripan ({method.capitalize()}): {similarity:.2f}\")\n",
    "            else:\n",
    "                print(f\"Gagal menghitung embedding menggunakan {method.capitalize()}.\")\n",
    "    else:\n",
    "        print(\"Wajah tidak terdeteksi pada salah satu gambar.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Klasifikasi Suku/Etnis\n",
    "\n",
    "Sekarang kita akan menggunakan model ResNet50 untuk memprediksi suku/etnis dari wajah yang terdeteksi. Pastikan model `ethnic_model.pth` sudah ada."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Daftar kelas suku/etnis (sesuaikan dengan dataset Anda)\n",
    "classes = ['Batak', 'China', 'Jawa', 'Minang', 'Nias', 'Papua', 'Sunda']\n",
    "\n",
    "# Inisialisasi dan muat model\n",
    "model = initialize_ethnic_model(len(classes))\n",
    "model_path = 'ethnic_model.pth'\n",
    "try:\n",
    "    model.load_state_dict(torch.load(model_path))\n",
    "    print(\"Model berhasil dimuat!\")\n",
    "except FileNotFoundError:\n",
    "    print(\"File model 'ethnic_model.pth' tidak ditemukan. Silakan latih model terlebih dahulu di aplikasi Streamlit.\")\n",
    "    model = None\n",
    "\n",
    "if model is not None and len(faces) > 0:\n",
    "    # Preprocessing gambar untuk klasifikasi\n",
    "    face = image[y:y+h, x:x+w]\n",
    "    image_tensor = preprocess_for_classification(face)\n",
    "\n",
    "    # Pindahkan ke perangkat yang sesuai\n",
    "    device = torch.device(\"cuda\" if torch.cuda.is_available() else \"cpu\")\n",
    "    model = model.to(device)\n",
    "    image_tensor = image_tensor.to(device)\n",
    "    model.eval()\n",
    "\n",
    "    # Prediksi\n",
    "    with torch.no_grad():\n",
    "        outputs = model(image_tensor)\n",
    "        probabilities = torch.softmax(outputs, dim=1).cpu().numpy()[0]\n",
    "        predicted_idx = np.argmax(probabilities)\n",
    "        predicted_suku = classes[predicted_idx]\n",
    "        confidence = probabilities[predicted_idx]\n",
    "\n",
    "    print(f\"Prediksi Suku: {predicted_suku}\")\n",
    "    print(f\"Confidence: {confidence:.4f}\")\n",
    "\n",
    "    # Visualisasi probabilitas\n",
    "    plt.figure(figsize=(8, 4))\n",
    "    plt.bar(classes, probabilities)\n",
    "    plt.xlabel('Suku/Etnis')\n",
    "    plt.ylabel('Probabilitas')\n",
    "    plt.title('Distribusi Probabilitas Suku/Etnis')\n",
    "    plt.show()\n",
    "else:\n",
    "    print(\"Prediksi suku tidak dapat dilakukan.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Deteksi Gender\n",
    "\n",
    "Sekarang kita akan menggunakan DeepFace untuk memprediksi gender dari wajah yang terdeteksi."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Pastikan gambar sudah dimuat dari langkah sebelumnya\n",
    "if image is None:\n",
    "    print(\"Gambar belum dimuat. Jalankan langkah sebelumnya terlebih dahulu.\")\n",
    "elif len(faces) == 0:\n",
    "    print(\"Tidak ada wajah yang terdeteksi pada gambar.\")\n",
    "else:\n",
    "    # Simpan gambar sementara untuk DeepFace\n",
    "    with tempfile.NamedTemporaryFile(delete=False, suffix='.jpg') as tmp_file:\n",
    "        cv2.imwrite(tmp_file.name, image)\n",
    "        image_path = tmp_file.name\n",
    "\n",
    "    try:\n",
    "        # Gunakan DeepFace untuk deteksi gender\n",
    "        result = DeepFace.analyze(\n",
    "            img_path=image_path,\n",
    "            actions=['gender'],\n",
    "            detector_backend='mtcnn',\n",
    "            enforce_detection=True\n",
    "        )\n",
    "        predicted_gender = result[0]['dominant_gender']\n",
    "        gender_scores = result[0]['gender']\n",
    "        probabilities = [gender_scores['Man'] / 100, gender_scores['Woman'] / 100]\n",
    "        confidence = max(probabilities)\n",
    "\n",
    "        print(f\"Prediksi Gender: {predicted_gender}\")\n",
    "        print(f\"Confidence: {confidence:.4f}\")\n",
    "\n",
    "        # Visualisasi probabilitas\n",
    "        plt.figure(figsize=(6, 4))\n",
    "        plt.bar(['Male', 'Female'], probabilities)\n",
    "        plt.xlabel('Gender')\n",
    "        plt.ylabel('Probabilitas')\n",
    "        plt.title('Distribusi Probabilitas Gender')\n",
    "        plt.show()\n",
    "\n",
    "    except Exception as e:\n",
    "        print(f\"Gagal mendeteksi gender: {e}\")\n",
    "    finally:\n",
    "        # Hapus file sementara\n",
    "        os.remove(image_path)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Kesimpulan\n",
    "\n",
    "Notebook ini menunjukkan cara menggunakan fungsi utama sistem:\n",
    "- **Deteksi Wajah**: Menggunakan MTCNN, RetinaFace, atau Haar Cascade untuk menemukan wajah dalam gambar.\n",
    "- **Face Similarity**: Membandingkan kemiripan wajah menggunakan Siamese Network, FaceNet, atau ArcFace.\n",
    "- **Klasifikasi Suku/Etnis**: Menggunakan model ResNet50 untuk memprediksi suku/etnis.\n",
    "- **Deteksi Gender**: Menggunakan DeepFace untuk memprediksi gender.\n",
    "\n",
    "Untuk penggunaan lebih lanjut, jalankan aplikasi Streamlit (`streamlit run app.py`) untuk mengakses semua fitur melalui antarmuka web."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "praktikum_env_new",
   "language": "python",
   "name": "praktikum_env_new"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}